<a href="https://colab.research.google.com/github/ncavasin/perceptron/blob/main/perceptron_cavasin_143501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 11089 - Sistemas Inteligentes - Universidad Nacional de Luján - 2021
# Perceptron
Crear y entrenar un perceptrón capaz de clasificar las entradas recibidas en base a una función linealmente separable:
- El perceptrón debe recibir tres entradas binarias.
- Los pesos finales y el valor de la función umbral no pueden definirse a priori sino que serán determinados por el entrenamiento y los ejemplos en él utilizados.
- La clasificación final sobre el conjunto de datos de entrada debe representarse gráficamente, en la forma de un plano que discrimine los valores de entrada en base a la función representada por el perceptrón.
- Explicar brevemente cada uno de los criterios adoptados durantes las etapas de codificación, entrenamiento y evaluación.
- El rango de cada coordenada debe ser [0, 1.5].

**Fecha de entrega: 08/05/2021**

## Resolución

### Paso 1 - Inicialización:
- Se definen los valores booleanos de las 3 entradas .
- Se define por cada combinación de entrada el valor de salida de la función lógica AND correspondiente.
- Se los combina en una matriz para facilitar su presentación.

In [2]:
import numpy as np
import pandas as pd

# Hardcodeo los valores de las tres entradas, pues no van a variar
x1 = pd.Series((0, 0, 0, 0, 1, 1, 1, 1), name='x1')
x2 = pd.Series((0, 0, 1, 1, 0, 0, 1, 1), name='x2')
x3 = pd.Series((0, 1, 0, 1, 0, 1, 0, 1), name='x3')

# Defino contador para luego mostrar en matriz
iter = 0

# Los combino en una matriz
matriz = pd.concat([x1, x2, x3], axis=1)

# Defino la funcion de salida, en este caso la funcion logica AND
y = pd.Series((0, 0, 0, 0, 0, 0, 0, 1), name='y')

# La inserto a la tabla
matriz = pd.concat([matriz, y], axis=1)

# Muestro la tabla
matriz

,x1,x2,x3,y
0,0,0,0,0
1,0,0,1,0
2,0,1,0,0
3,0,1,1,0
4,1,0,0,0
5,1,0,1,0
6,1,1,0,0
7,1,1,1,1


### Paso 2 - Funciones
Se definen las funciones auxiliares ``train()``, ``adjust()`` y ``sep_function()``  para construir el Perceptrón.



___


``sep_function()``:

Esta función es aplicada a cada Z obtenido para acotar la salida al rango de valores binarios $[0, 1]$.

In [3]:
def sep_function(z):
    """
    Esta funcion representa la separacion lineal de cada salida obtenida.
    Si z >= 0.5 ==> z = 1
    Si z <  0.5 ==> z = 0
    """

    if z >= 0.5:
        return 1
    else:
        return 0

___

``adjust()``:

Cada vez que el valor Z sea diferente al valor Y -correspondiente a la salida correcta para dicha combinación de entradas- se aplica esta función cuyo objetivo es calcular el error de estimación para poder ajustar $b$ (el threshold) y $w_i$ (los pesos asignados).

In [4]:
def adjust(z, y, t, p, lr):
    
    # Calculo error
    error = y - z

    # Ajusto threhsold
    t = t + (-1 *(lr * error))

    # Ajusto pesos
    p['x1'] += (p['x1'] * lr * error)
    p['x2'] += (p['x2'] * lr * error)
    p['x3'] += (p['x3'] * lr * error)

    return t, p

___

``train()``:

Se alimenta de las entradas definidas en la matriz y para cada combinacion de ellas calcula un valor de Z al que posteriormente aplica la función de separación para redondear su valor.

La fórmula utilizada es para obtener Z es $Z=(\sum_{i=1}^n x_i*w_i) + b$, siendo: 

- $n$ la cantidad de entradas en la matriz.
- $x_i$ el valor de la entrada procesada.
- $w_i$ el peso asginado una entrada.
- $b$ el threshold.

In [5]:
def train(mat, p, t, lr, e):

    errores = True

    while errores and e < 100:
        errores = False

        # Creo lista para almacenar valores calculados
        z2 = []

        for i in range(0, matriz.shape[0]):

            # Calculo z
            z = ((mat['x1'][i] * p['x1']) + (mat['x2'][i] * p['x2']) + (mat['x3'][i] * p['x3'])) - t

            # Aplico la funcion de separacion
            z = sep_function(z)

            # Verifico si hubo error
            if z != mat['y'][i]:
                errores = True

                # Ajusto
                t, p = adjust(z, mat['y'][i], t, p, lr)
                
                # Incremento contador de iteraciones
                e += 1     
            
            # Agrego el valor calculado a la lista
            z2.append(z)
    if epochs == 100:
        exit(-1)
    else:
        return t, p, e, z2

### Paso 3 - Ejecucion

- Se inicializan los pesos de manera aleatoria
- Se define un $b$ ``threshold`` = 0.5
- Se define la velocidad de aprendizaje ``learning_rate`` = 0.2
- Se inicializa en 0 el acumulador ``epochs`` que indicará cuantas veces se ajustaron los pesos.


**Nota:** el límite superior de ajustes de peso permitido es *100* para evitar demoras de procesamiento. Cuando sea alcanzado, se retornaran todos los valores actuales aunque tengan errores de clasificación y esto se podrá ver reflejado en la matriz de resultados.

In [9]:
import random 

# Defino los valores iniciales del threshold, learn rate y epochs
threshold = 0.5
learn_rate = 0.2
epochs = 0

# Inicializo pesos al azar
pesos = {}
pesos['x1'] = np.float(round(random.uniform(0, 1.5), 2))
pesos['x2'] = np.float(round(random.uniform(0, 1.5), 2))
pesos['x3'] = np.float(round(random.uniform(0, 1.5), 2))


print('PARAMETROS INICIALES:')
print(f'Epochs = {epochs}')
print(f'Threshold = {threshold:.2f}')
print(f"Vel aprendizaje = {learn_rate}")
for index, value in enumerate(pesos):
    print(f'Peso {index} = {pesos[value]:.4f}')


print('\n<=== INI PERIODO DE ENTRENAMIENTO ===>')
# Entreno
threshold, pesos, epochs, z = train(matriz, pesos, threshold, learn_rate, epochs)

# Formateo para mostrar
z = pd.Series(np.asarray(z), name=f'z{iter}')
matriz = pd.concat([matriz, z], axis= 1)

# Aumento el contador de veces ejecutadas del algoritmo
iter += 1
print('<=== FIN PERIODO DE ENTRENAMIENTO ===>')


print('\nPARAMETROS FINALES:')
print(f'Epochs = {epochs}')
print(f'Threshold = {threshold:.2f}')
print(f"Vel aprendizaje = {learn_rate}")
for index, value in enumerate(pesos):
    print(f'Peso {index} = {pesos[value]:.4f}')

print('\n\nMatriz original con los Z calculados:')
matriz

PARAMETROS INICIALES:
Epochs = 0
Threshold = 0.50
Vel aprendizaje = 0.2
Peso 0 = 1.1000
Peso 1 = 0.3400
Peso 2 = 1.4800

<=== INI PERIODO DE ENTRENAMIENTO ===>
<=== FIN PERIODO DE ENTRENAMIENTO ===>

PARAMETROS FINALES:
Epochs = 12
Threshold = 0.90
Vel aprendizaje = 0.2
Peso 0 = 0.5740
Peso 1 = 0.1774
Peso 2 = 0.7723


Matriz original con los Z calculados:


,x1,x2,x3,y,z0,z1,z2,z3
0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0
5,1,0,1,0,0,0,0,0
6,1,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1


### Paso 4 - Representacion grafica

Como resultado del entrenamiento del perceptron se grafica en R3 el plano Z obtenido en base a los valores calculados junto con los puntos de entrada.

Recordar que en R3 la ecuación general de una recta es $0 = aX + bY + cZ + d$. La equivalencia, utilizando la nomenclatura del código, es la siguiente:
- $a$ = $peso_1$
- $b$ = $peso_2$
- $c$ = $peso_3$
- $d$ = ``threshold``

Despejando, se obtiene $Z = \frac{aX + bY + D}{c}$. 

En nuestra nomenclatura quedaría:
$Z = \frac{peso_1 * X + peso_2 * Y + threshold}{peso_3}$

In [ ]:
import plotly.graph_objects as go

xx = np.linspace(-1,1,10)
yy = np.linspace(-1,1,10)
X,Y = np.meshgrid(xx,yy)

Z = (pesos['x1'] * X + pesos['x2'] * Y + threshold) / pesos['x3']

fig = go.Figure(data=[go.Surface(x=X, y=Y,z=Z, colorscale=['cyan', 'cyan', 'cyan']),   
                      go.Scatter3d(x=matriz['x1'], y=matriz['x2'], z=matriz['x3'],
                                   mode='markers', marker=dict(size=4, color='green'))
                      ]
                )

fig.update_layout(width=800, height=800)
fig.show()
